<a href="https://colab.research.google.com/github/Rattikorn10192/Project_Analytic_Dashboard/blob/main/MAD7101_FinalPRJ_JINGJAI_PHAMACY_20231209final_oiljai.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MAD7101 Final Project -- Jingjai Phamacy

## Member

* 6420422011 JUNTANEE PATTANASUKKUL
* 6420422011 JUNTANEE PATTANASUKKUL

# **0. Library**

In [110]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
# Authenticate and create the PyDrive client.

auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter
import seaborn as sns
import math
from termcolor import colored

In [111]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive

link_file = 'https://drive.google.com/file/d/1Kwhx3rf8PPYku_TdRZzVPHGyTKwjAem5/view?usp=drive_link'
_, _, _, _, _, id, _ = link_file.split('/')
downloaded = drive.CreateFile({'id': id})
downloaded.GetContentFile('Actual_Target.csv')



In [76]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 1. DATA

In [77]:
!pip install -U accelerate
!pip install -U transformers
!pip install -U pandasql
!pip install SQLAlchemy==1.4.46
! pip install termcolor

In [78]:
data_df = pd.read_csv('/content/drive/MyDrive/Target_Actual.csv')
data_df

,month,sale_target,total_actual,%Achieve
0,Jan,"5,894,729.03","5,319,276.20",90%
1,Feb,"5,319,276.20","4,612,196.35",87%
2,Mar,"4,612,196.35","4,737,132.03",103%
3,Apr,"4,737,132.03","5,029,716.80",106%
4,May,"5,029,716.80","5,208,796.14",104%
5,Jun,"4,768,653.16","5,020,333.92",105%
6,Jul,"5,020,333.92","4,720,628.90",94%
7,Aug,"4,720,628.90","4,545,094.96",96%
8,Sep,"4,545,094.96","4,879,536.02",107%
9,Oct,"4,879,536.02","4,373,159.99",90%


In [112]:
link_file = 'https://drive.google.com/file/d/1XOiTyAT28GU79K7rb0fKvLz_0JOUGIc7/view?usp=drive_link'
_,_,_,_,_,id,_ = link_file.split('/')
downloaded = drive.CreateFile({'id':id})
downloaded.GetContentFile('Phama_Data.csv')

In [79]:
# Specify the path to your CSV file in your Google Drive
csv_file_path = '/content/drive/MyDrive/summary_sale_pure2023.csv'

# Read the CSV file
df = pd.read_csv('/content/drive/MyDrive/summary_sale_pure2023.csv')
df.head()


,SITE_NO,SITE_NAME,DATE_SALE,ART_NO,ART_NAME,ART_SV_NO,BRAND,DIVISION_NAME,DEPARTMENT_NAME,SUBDEPARTMENT_NAME,SALE_AMT,SALE_QTY,COGS_AMT,GP_AMT,PRICE
0,41016,Sapankwai,2023-08-02,102482069,P_BENZAC AC GEL 2.5% 15G.,0,CETAPHIL,06 PHARMACY,0610 PRESCRIPTION,061020 ANTIBIOTIC,142.056,1.0,99.336,42.720,142.056000
1,41003,Suksawat,2023-06-13,102167170,P_เกร๊ทเตอร์ คาอาบอนชาร์โคล10แคปซูล,0,GREATER,06 PHARMACY,0620 OVER THE COUNTER,062050 GASTROINTESTINALS SYSTEM,46.729,2.0,28.400,18.329,23.364500
2,41005,Samrong,2023-05-21,100653482,P_ทิฟฟี่เดย์ 4 เม็ด,0,TIFFY,06 PHARMACY,0610 PRESCRIPTION,061080 RESPIRATORIES SYSTEM,54.206,6.0,42.600,11.606,9.034333
3,41016,Sapankwai,2023-11-22,103002110,P_ดีโมซานา แคลเซียม 20 เม็ด,0,DEMOSANA,06 PHARMACY,0620 OVER THE COUNTER,062080 VITAMINS&MINERALS,135.514,1.0,94.860,40.654,135.514000
4,41013,Tiwanon,2023-08-10,100652257,P_PROCTOSEDYL OINT. 15 GM,0,SANOFI-AVENTIS,06 PHARMACY,0610 PRESCRIPTION,061030 CARDIOVASCULARS SYSTEM,186.916,1.0,156.640,30.276,186.916000


## Total Sales Performance by Brand

In [113]:
df_total = df.loc[:,['SITE_NAME','SALE_AMT']]
df_total_grouped = df_total.groupby('SITE_NAME')['SALE_AMT'].sum().reset_index()
df_total_grouped['%Sale_amt'] = (df_total_grouped['SALE_AMT'] / df_total_grouped['SALE_AMT'].sum()) *100
df_total_grouped['%Sale_amt'] = df_total_grouped['%Sale_amt'].apply(lambda x: math.floor(x))
df_total_grouped = df_total_grouped.sort_values(by='SALE_AMT', ascending=False)
df_total_grouped.reset_index(drop=True, inplace=True)

df_total_grouped

,SITE_NAME,SALE_AMT,%Sale_amt
0,Sapankwai,1.377009e+07,25
1,Samrong,1.124410e+07,21
2,Suksawat,1.120053e+07,21
3,Rama 2,8.722225e+06,16
4,Tiwanon,8.277574e+06,15


In [114]:
df_2 = df.loc[:,['SITE_NAME','DEPARTMENT_NAME','SALE_AMT']]
df_2 = df_2.groupby(['SITE_NAME', 'DEPARTMENT_NAME'])['SALE_AMT'].sum().reset_index()
df_2['%Sale_amt'] = (df_2['SALE_AMT'] / df_2['SALE_AMT'].sum()) *100
df_2['%Sale_amt'] = df_2['%Sale_amt'].apply(lambda x: math.floor(x))

df_2.reset_index(drop=True, inplace=True)

df_2

,SITE_NAME,DEPARTMENT_NAME,SALE_AMT,%Sale_amt
0,Rama 2,0461 FACE & COSMETICS,176.542,0
1,Rama 2,0610 PRESCRIPTION,1312734.875,2
2,Rama 2,0620 OVER THE COUNTER,5793425.492,10
3,Rama 2,0630 WELL BEING,1250214.692,2
4,Rama 2,0640 HERBAL PREPARATION,365673.769,0
5,Samrong,0461 FACE & COSMETICS,176.636,0
6,Samrong,0610 PRESCRIPTION,1920007.032,3
7,Samrong,0620 OVER THE COUNTER,7662451.249,14
8,Samrong,0630 WELL BEING,1299271.089,2
9,Samrong,0640 HERBAL PREPARATION,362198.788,0


In [115]:
# Assuming df_2 is your DataFrame
df_2 = df.loc[:, ['SITE_NAME', 'DEPARTMENT_NAME', 'SALE_AMT']]
df_2_grouped = df_2.groupby(['SITE_NAME', 'DEPARTMENT_NAME'])['SALE_AMT'].sum().reset_index()
df_2_grouped['%Sale_amt'] = (df_2_grouped['SALE_AMT'] / df_2_grouped['SALE_AMT'].sum()) * 100
df_2_grouped['%Sale_amt'] = df_2_grouped['%Sale_amt'].apply(lambda x: math.floor(x))
df_2_grouped.reset_index(drop=True, inplace=True)

# Create a pivot table for the double nested pie chart
df_2_pivot = df_2_grouped.pivot_table(values='%Sale_amt', index='SITE_NAME', columns='DEPARTMENT_NAME', aggfunc='sum', fill_value=0)
df_2_pivot

DEPARTMENT_NAME,0461 FACE & COSMETICS,0610 PRESCRIPTION,0620 OVER THE COUNTER,0630 WELL BEING,0640 HERBAL PREPARATION
SITE_NAME,,,,,
Rama 2,0,2,10,2,0
Samrong,0,3,14,2,0
Sapankwai,0,3,16,4,0
Suksawat,0,4,13,2,0
Tiwanon,0,2,10,1,0


## 1.1Total Sales Performance by Sapankwai Branch

In [116]:
# Assuming df is your DataFrame
df_3 = df.loc[:, ['SITE_NAME', 'BRAND', 'SALE_AMT', 'SALE_QTY', 'PRICE']]

# Filter for SITE_NAME = 'Sapankwai'
df_3_sapankwai = df_3[df_3['SITE_NAME'] == 'Sapankwai']

# Group by 'BRAND' and calculate the sum of 'SALE_AMT'
df_3_grouped_sapankwai = df_3_sapankwai.groupby(['BRAND']).agg({
    'SALE_AMT': 'sum',
    'SALE_QTY': 'sum',
}).reset_index()

# Calculate the mean of PRICE as SALE_AMT / SALE_QTY
df_3_grouped_sapankwai['PRICE'] = df_3_grouped_sapankwai['SALE_AMT'] / df_3_grouped_sapankwai['SALE_QTY']

# Sort by 'SALE_AMT' in descending order
df_3_grouped_sapankwai = df_3_grouped_sapankwai.sort_values(by='SALE_AMT', ascending=False)

# Get the top 10 brands
top_10_brands = df_3_grouped_sapankwai.nlargest(10, 'SALE_AMT')

# Display the top 10 brands
top_10_brands


,BRAND,SALE_AMT,SALE_QTY,PRICE
293,OTHER,826333.852,11438.0,72.244610
125,EUCERIN,778810.977,1103.0,706.084295
60,BLACKMORES,638219.781,1343.0,475.219494
226,MEGA,634166.094,2505.0,253.160117
419,TIGER,500603.229,5324.0,94.027654
94,COUNTERPAIN,393370.195,3159.0,124.523645
444,VISTRA,302646.011,1195.0,253.260260
324,POY SEAN,215894.440,3685.0,58.587365
80,CERAVE,202372.680,471.0,429.665987
377,SIANGPURE,183194.259,2801.0,65.403163


In [117]:
# Assuming df is your DataFrame
df_3 = df.loc[:, ['SITE_NAME', 'BRAND', 'SALE_AMT', 'SALE_QTY', 'PRICE']]
df_3_grouped = df_3.groupby(['SITE_NAME', 'BRAND']).agg({
    'SALE_AMT': 'sum',
    'SALE_QTY': 'sum',
}).reset_index()

# Calculate the mean of PRICE as SALE_AMT / SALE_QTY
df_3_grouped['PRICE'] = df_3_grouped['SALE_AMT'] / df_3_grouped['SALE_QTY']

# Filter for SITE_NAME = 'Sapankwai'
df_3_grouped_sapankwai = df_3_grouped[df_3_grouped['SITE_NAME'] == 'Sapankwai']

df_3_grouped_sapankwai = df_3_grouped_sapankwai.sort_values(by=['BRAND','SALE_AMT'], ascending=False)

# Display the grouped DataFrame for SITE_NAME = 'Sapankwai'
df_3_grouped_sapankwai

,SITE_NAME,BRAND,SALE_AMT,SALE_QTY,PRICE
1372,Sapankwai,ZYRTEC,34934.561,285.0,122.577407
1371,Sapankwai,ZION,15767.445,300.0,52.558150
1370,Sapankwai,ZENSIL,6542.046,200.0,32.710230
1369,Sapankwai,ZEMA,2927.113,87.0,33.644977
1368,Sapankwai,ZEALOSE,1084.112,8.0,135.514000
...,...,...,...,...,...
913,Sapankwai,A BONNE',570.187,14.0,40.727643
912,Sapankwai,91E,2444.870,39.0,62.688974
911,Sapankwai,8 CC HERBAL OIL,6065.420,17.0,356.789412
910,Sapankwai,5PAGODAS,27246.685,1069.0,25.488012


## 1.2Total Sales Performance by Samrong Branch

In [118]:
# Assuming df is your DataFrame
df_4 = df.loc[:, ['SITE_NAME', 'BRAND', 'SALE_AMT', 'SALE_QTY', 'PRICE']]

# Filter for SITE_NAME = 'Samrong'
df_4_samrong = df_4[df_4['SITE_NAME'] == 'Samrong']

# Group by 'BRAND' and calculate the sum of 'SALE_AMT'
df_4_grouped_samrong = df_4_samrong.groupby(['BRAND']).agg({
    'SALE_AMT': 'sum',
    'SALE_QTY': 'sum',
}).reset_index()

# Calculate the mean of PRICE as SALE_AMT / SALE_QTY
df_4_grouped_samrong['PRICE'] = df_4_grouped_samrong['SALE_AMT'] / df_4_grouped_samrong['SALE_QTY']

# Sort by 'SALE_AMT' in descending order
df_4_grouped_samrong = df_4_grouped_samrong.sort_values(by='SALE_AMT', ascending=False)

# Get the top 10 brands
top_10_brands_samrong = df_4_grouped_samrong.nlargest(10, 'SALE_AMT')

# Display the top 10 brands
top_10_brands_samrong


,BRAND,SALE_AMT,SALE_QTY,PRICE
288,OTHER,839844.492,12513.0,67.117757
221,MEGA,637150.610,2956.0,215.544861
59,BLACKMORES,604060.337,1334.0,452.818843
257,NATWELL,571019.617,2009.0,284.230770
143,G-LUCKY MASK,243361.729,2706.0,89.934120
439,VISTRA,220386.846,945.0,233.213594
122,EUCERIN,196509.913,306.0,642.189258
448,YAZ,177767.227,464.0,383.119024
92,COUNTERPAIN,146444.949,1247.0,117.437810
325,PROPOLIZ,133394.941,1326.0,100.599503


## 1.3Total Sales Performance by Suksawat Branch

In [119]:
# Assuming df is your DataFrame
df_5 = df.loc[:, ['SITE_NAME', 'BRAND', 'SALE_AMT', 'SALE_QTY', 'PRICE']]

# Filter for SITE_NAME = 'Suksawat'
df_5_suksawat = df_5[df_5['SITE_NAME'] == 'Suksawat']

# Group by 'BRAND' and calculate the sum of 'SALE_AMT'
df_5_grouped_suksawat = df_5_suksawat.groupby(['BRAND']).agg({
    'SALE_AMT': 'sum',
    'SALE_QTY': 'sum',
}).reset_index()

# Calculate the mean of PRICE as SALE_AMT / SALE_QTY
df_5_grouped_suksawat['PRICE'] = df_5_grouped_suksawat['SALE_AMT'] / df_5_grouped_suksawat['SALE_QTY']

# Sort by 'SALE_AMT' in descending order
df_5_grouped_suksawat = df_5_grouped_suksawat.sort_values(by='SALE_AMT', ascending=False)

# Get the top 10 brands
top_10_brands_suksawat = df_5_grouped_suksawat.nlargest(10, 'SALE_AMT')

# Display the top 10 brands
top_10_brands_suksawat


,BRAND,SALE_AMT,SALE_QTY,PRICE
287,OTHER,874655.711,14235.0,61.444026
221,MEGA,646232.843,2937.0,220.031612
56,BLACKMORES,356417.281,791.0,450.590747
121,EUCERIN,311362.963,508.0,612.919219
256,NATWELL,248582.315,878.0,283.123366
143,G-LUCKY MASK,221384.059,2438.0,90.805603
435,VISTRA,190553.123,765.0,249.089050
290,P.M.L.,173413.166,2766.0,62.694565
0,3M,167984.849,1763.0,95.283522
208,LOC,158446.275,1613.0,98.230797


## 1.4Total Sales Performance by Rama2 Branch

In [120]:
# Assuming df is your DataFrame
df_6 = df.loc[:, ['SITE_NAME', 'BRAND', 'SALE_AMT', 'SALE_QTY', 'PRICE']]

# Filter for SITE_NAME = 'Rama 2'
df_6_rama2 = df_6[df_6['SITE_NAME'] == 'Rama 2']

# Group by 'BRAND' and calculate the sum of 'SALE_AMT'
df_6_grouped_rama2 = df_6_rama2.groupby(['BRAND']).agg({
    'SALE_AMT': 'sum',
    'SALE_QTY': 'sum',
}).reset_index()

# Calculate the mean of PRICE as SALE_AMT / SALE_QTY
df_6_grouped_rama2['PRICE'] = df_6_grouped_rama2['SALE_AMT'] / df_6_grouped_rama2['SALE_QTY']

# Sort by 'SALE_AMT' in descending order
df_6_grouped_rama2 = df_6_grouped_rama2.sort_values(by='SALE_AMT', ascending=False)

# Get the top 10 brands
top_10_brands_rama2 = df_6_grouped_rama2.nlargest(10, 'SALE_AMT')

# Display the top 10 brands
top_10_brands_rama2


,BRAND,SALE_AMT,SALE_QTY,PRICE
220,MEGA,936646.420,4038.0,231.958004
287,OTHER,668182.294,10383.0,64.353491
120,EUCERIN,251798.114,370.0,680.535443
59,BLACKMORES,249182.419,558.0,446.563475
142,G-LUCKY MASK,190402.831,2322.0,81.999497
90,COUNTERPAIN,140352.502,1324.0,106.006421
433,VISTRA,138780.240,570.0,243.474105
409,TIGER,128110.007,1966.0,65.162771
206,LOC,119248.662,1022.0,116.681665
430,VIDA,101080.630,831.0,121.637341


## 1.5Total Sales Performance by Tiwanon Branch

In [121]:
# Assuming df is your DataFrame
df_7 = df.loc[:, ['SITE_NAME', 'BRAND', 'SALE_AMT', 'SALE_QTY', 'PRICE']]

# Filter for SITE_NAME = 'Tiwanon'
df_7_tiwanon = df_7[df_7['SITE_NAME'] == 'Tiwanon']

# Group by 'BRAND' and calculate the sum of 'SALE_AMT'
df_7_grouped_tiwanon = df_7_tiwanon.groupby(['BRAND']).agg({
    'SALE_AMT': 'sum',
    'SALE_QTY': 'sum',
}).reset_index()

# Calculate the mean of PRICE as SALE_AMT / SALE_QTY
df_7_grouped_tiwanon['PRICE'] = df_7_grouped_tiwanon['SALE_AMT'] / df_7_grouped_tiwanon['SALE_QTY']

# Sort by 'SALE_AMT' in descending order
df_7_grouped_tiwanon = df_7_grouped_tiwanon.sort_values(by='SALE_AMT', ascending=False)

# Get the top 10 brands
top_10_brands_tiwanon = df_7_grouped_tiwanon.nlargest(10, 'SALE_AMT')

# Display the top 10 brands
top_10_brands_tiwanon


,BRAND,SALE_AMT,SALE_QTY,PRICE
278,OTHER,685485.427,10069.0,68.078799
214,MEGA,461101.452,1809.0,254.893008
58,BLACKMORES,408469.702,899.0,454.360069
423,VISTRA,177595.700,704.0,252.266619
139,G-LUCKY MASK,176000.754,2023.0,86.999878
119,EUCERIN,172759.339,278.0,621.436471
420,VIDA,130441.126,1073.0,121.566753
0,3M,116842.627,1231.0,94.916838
201,LOC,116352.547,1665.0,69.881410
317,PROPOLIZ,109617.716,950.0,115.387069


# 2. DATA --Help

In [122]:
import pandasql as ps
query = """SELECT * FROM df limit 5"""
ps.sqldf(query, globals())

,SITE_NO,SITE_NAME,DATE_SALE,ART_NO,ART_NAME,ART_SV_NO,BRAND,DIVISION_NAME,DEPARTMENT_NAME,SUBDEPARTMENT_NAME,SALE_AMT,SALE_QTY,COGS_AMT,GP_AMT,PRICE
0,41016,Sapankwai,2023-08-02,102482069,P_BENZAC AC GEL 2.5% 15G.,0,CETAPHIL,06 PHARMACY,0610 PRESCRIPTION,061020 ANTIBIOTIC,142.056,1.0,99.336,42.720,142.056000
1,41003,Suksawat,2023-06-13,102167170,P_เกร๊ทเตอร์ คาอาบอนชาร์โคล10แคปซูล,0,GREATER,06 PHARMACY,0620 OVER THE COUNTER,062050 GASTROINTESTINALS SYSTEM,46.729,2.0,28.400,18.329,23.364500
2,41005,Samrong,2023-05-21,100653482,P_ทิฟฟี่เดย์ 4 เม็ด,0,TIFFY,06 PHARMACY,0610 PRESCRIPTION,061080 RESPIRATORIES SYSTEM,54.206,6.0,42.600,11.606,9.034333
3,41016,Sapankwai,2023-11-22,103002110,P_ดีโมซานา แคลเซียม 20 เม็ด,0,DEMOSANA,06 PHARMACY,0620 OVER THE COUNTER,062080 VITAMINS&MINERALS,135.514,1.0,94.860,40.654,135.514000
4,41013,Tiwanon,2023-08-10,100652257,P_PROCTOSEDYL OINT. 15 GM,0,SANOFI-AVENTIS,06 PHARMACY,0610 PRESCRIPTION,061030 CARDIOVASCULARS SYSTEM,186.916,1.0,156.640,30.276,186.916000


In [123]:
from termcolor import colored

In [124]:
# Specify the path to your CSV file in your Google Drive
csv_file_path = '/content/drive/MyDrive/art_search.csv'

# Read the CSV file
#df_art = pd.read_csv('/content/drive/MyDrive/art_search.csv')
df_art = pd.read_csv('/content/drive/MyDrive/art_perunit_results.csv')
df_art.head()

,ART_NO,ART_NAME,SALE_AMT,COGS_AMT,GP_AMT,SALE_UNIT,COGS_UNIT,GP_UNIT
0,100510827,TVเทนโซพลาสติคการ์ตูน(MP),142.056,102.360,39.696,35.514000,25.590000,9.924000
1,100510828,P_TVเทนโซพลาสติคสีเนื้อ20'S,9476.568,6815.913,2660.655,36.730884,26.418267,10.312616
2,100510829,P_TVเทนโซพลาสติคใส20ชิ้น(MP),7869.112,5592.506,2276.606,37.118453,26.379745,10.738708
3,100617595,P_TVถุงยางดูเร็กเฟเธอร์ไลท์(MP,4924.288,3191.508,1732.780,87.933714,56.991214,30.942500
4,100617597,P_TVดูเร็กซ์สตรอเบอรี่ชมพู(MP),7708.432,5569.348,2139.084,78.657469,56.830082,21.827388


In [125]:
df_art.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3559 entries, 0 to 3558
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   ART_NO     3559 non-null   int64  
 1   ART_NAME   3559 non-null   object 
 2   SALE_AMT   3559 non-null   float64
 3   COGS_AMT   3559 non-null   float64
 4   GP_AMT     3559 non-null   float64
 5   SALE_UNIT  3559 non-null   float64
 6   COGS_UNIT  3559 non-null   float64
 7   GP_UNIT    3559 non-null   float64
dtypes: float64(6), int64(1), object(1)
memory usage: 222.6+ KB


# Line Bot SDK

In [127]:
!pip install pyngrok

In [94]:
!pip install line-bot-sdk

In [128]:
valid_art = df_art['ART_NO'].values.tolist()

In [ ]:
df_art.dtypes

## 2. Response Chatbot

In [132]:
import json
import os
from flask import Flask, request, make_response, abort
from pyngrok import ngrok
import pandas as pd
import pandasql as ps
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from linebot import LineBotApi
from linebot.models import FlexSendMessage, TextSendMessage
from linebot.v3.messaging import MessagingApi

from linebot import *
from linebot.models import *

port_no = 5000

#Flask


app = Flask(__name__)
ngrok.set_auth_token("2YJ3DnJI6zV3WvGP45s2M3Ennb6_5xjUQ8xgDFyN7AZHyLNat") # ngrok ของตัวเอง
public_url =  ngrok.connect(port_no).public_url

 # ของ Agent
line_bot_api = LineBotApi('Z0Qqy1sWUMw5RNKsaChGQKLSHwkivRGSKr4KYKFbcGp8TLb96q2TBQ6ycIqp937tG//hpOFL8qwRqMUlZE86XEkPg5f4SU59Wa92xMD4bZVPiPdYbwMaFwTOSA4yF0Bz0N4SiEZR99iOCtKAAR3w8wdB04t89/1O/w1cDnyilFU=')
handler = WebhookHandler('2ace8bbe1a2dc37fde3cb4593d12c1b8')

@app.route("/callback", methods=['POST'])
def callback():
    req = request.get_json(silent=True, force=True)
    intent_name = req["queryResult"]["intent"]["displayName"]

    text = req['originalDetectIntentRequest']['payload']['data']['message']['text']
    reply_token = req['originalDetectIntentRequest']['payload']['data']['replyToken']
    id = req['originalDetectIntentRequest']['payload']['data']['source']['userId']
    disname = line_bot_api.get_profile(id).display_name

    print('id = ' + id)
    print('name = ' + disname)
    print('text = ' + text)
    print('intent = ' + intent_name)
    print('reply_token = ' + reply_token)

    if intent_name == "jingjai summary - custom":
        reply_viz(text, reply_token)
    elif intent_name == "jingjai qa - custom":
        reply_report(text, reply_token)
    elif intent_name == "jingjai Target-Actual-custom":
        response_data = query_target_actual(text)
        print("Response Data:", response_data)
        if response_data is not None:
            # Generate the Flex Message content
            flex_message_content = create_target_actual_flex_message(response_data)

            # Print the Flex Message content for debugging
            print("Flex Message Content:", flex_message_content)

            # Use FlexSendMessage to create the Flex Message
            flex_message = FlexSendMessage(
                alt_text="Target vs Actual",
                contents=flex_message_content['contents']
            )

            # Print the complete FlexSendMessage for debugging
            print("Complete FlexSendMessage:", flex_message)

            # Use the LineBotApi to send the message
            line_bot_api.reply_message(reply_token, flex_message)
        else:
            line_bot_api.reply_message(reply_token, TextSendMessage(text="No data found for the specified month"))

    elif intent_name == "jingjai search - custom":
        response_data = query_artl(text)
        print("Response Data:", response_data)
        if response_data is not None:
            # Generate the Flex Message content
            flex_message_content = art_flex_message(response_data)

            # Print the Flex Message content for debugging
            print("Flex Message Content:", flex_message_content)

            # Use FlexSendMessage to create the Flex Message
            flex_message = FlexSendMessage(
                alt_text="Article Search",
                contents=flex_message_content['contents']
            )

            # Print the complete FlexSendMessage for debugging
            print("Complete FlexSendMessage:", flex_message)

            # Use the LineBotApi to send the message
            line_bot_api.reply_message(reply_token, flex_message)

        else:
            line_bot_api.reply_message(reply_token, TextSendMessage(text="No data found for the specified "))

    return "OK"

# Summary Report Part

def query_target_actual(month):
    # Valid months list
    valid_months = ["jan", "feb", "mar", "apr", "may", "jun", "jul", "aug", "sep", "oct", "nov", "dec","total"]

    # Convert the month variable to lowercase
    month_lower = month.lower()

    # Check if the lowercase month is in the valid months list
    if month_lower not in valid_months:
        return None  # Return early if the month is not valid

    # Filter the DataFrame based on the provided lowercase month
    result = data_df[data_df['month'].str.lower() == month_lower]

    # Return None if no records are found
    if result.empty:
        return None

    # Convert the result to a dictionary and return the first record
    return result.to_dict(orient='records')[0]



# Function to create Flex Message for Target-Actual data
def create_target_actual_flex_message(data_df):
    # Extract values from the response data
    month = data_df.get('month', 'N/A')
    sales_target = data_df.get(' sale_target ', 'N/A')
    total_actual = data_df.get(' total_actual ', 'N/A')
    percentage_achieve = data_df.get('%Achieve', 'N/A')

    # Calculate % achievement if both sales_target and total_actual are available
    if sales_target != 'N/A' and total_actual != 'N/A':
        percentage_achieve = (float(total_actual.replace(',', '')) / float(sales_target.replace(',', ''))) * 100
        percentage_achieve = f"{percentage_achieve:.2f}%"

    flex_message = {
        "type": "flex",
        "altText": "Target vs Actual",
        "contents": {
            "type": "bubble",
            "hero": {
                "type": "image",
                "url": "https://img.freepik.com/free-vector/hand-drawn-business-strategy_23-2149164272.jpg",
                "size": "full",
                "aspectRatio": "20:13",
                "aspectMode": "cover",
            },
            "body": {
                "type": "box",
                "layout": "vertical",
                "contents": [
                    {"type": "text", "text": f"Target vs Actual: {month}", "weight": "bold", "size": "xl"},
                    {"type": "text", "text": f"Sales Target: {sales_target}", "margin": "md"},
                    {"type": "text", "text": f"Total Actual: {total_actual}", "margin": "md"},
                    {"type": "text", "text": f"% Achieve: {percentage_achieve}", "margin": "md"},
                    # Add other elements here
                ],
            },
            # Add other elements here
        },
    }

    return flex_message

# ART
def query_artl(art_no):
    # Valid art numbers list
    valid_art = df_art['ART_NO'].values.tolist()

    # Check if art_no is in the valid_art list
    if int(art_no) not in valid_art:
        return None  # Return early if art_no is not valid
        print("Query Result:", int(art_no))

    try:
        # Filter the DataFrame based on the provided art_no
        result = df_art[df_art['ART_NO'] == int(art_no)]  # Use uppercase 'ART_NO'

        # Print the result for debugging
        print("Query Result:", result)

        # Return None if no records are found
        if result.empty:
            return None

        # Convert the result to a dictionary and return the first record
        return result.to_dict(orient='records')[0]
    except KeyError as e:
        print(f"Error: {e}. The 'ART_NO' column does not exist in the DataFrame.")
        return None

        # Convert the result to a dictionary and return the first record
        return result.to_dict(orient='records')[0]
    except KeyError as e:
        print(f"Error: {e}. The 'art_no' column does not exist in the DataFrame.")
        return None

    # Convert the result to a dictionary and return the first record
    return result.to_dict(orient='records')[0]

def art_flex_message(df_art):
    # Extract values from the response data
    ART_NO = df_art.get('ART_NO', 'N/A')
    ART_NAME = df_art.get('ART_NAME', 'N/A')  # Assuming the column name is 'art_name'
    SALE_UNIT = df_art.get('SALE_UNIT', 'N/A')
    COGS_UNIT = df_art.get('COGS_UNIT', 'N/A')
    GP_UNIT = df_art.get('GP_UNIT', 'N/A')

    flex_message = {
        "type": "flex",
        "altText": "Article Search",
        "contents": {
            "type": "bubble",
            "hero": {
                "type": "image",
                "url": "https://img.freepik.com/free-photo/medicine-container-spills-colorful-pills-background-generative-ai_188544-12527.jpg?t=st=1702036510~exp=1702040110~hmac=b6de9b686a1507403f555ea2da36d0445139c52f7fb14d1a8eae58e3759bf07b&w=1380",
                "size": "full",
                "aspectRatio": "20:13",
                "aspectMode": "cover",
            },
            "body": {
                "type": "box",
                "layout": "vertical",
                "contents": [
                    {"type": "text", "text": f"Article No: {ART_NO}", "weight": "bold", "size": "xl"},
                    {"type": "text", "text": f"Article Name: {ART_NAME}", "margin": "md"},
                    {"type": "text", "text": f"Price : $ {SALE_UNIT:.2f}", "margin": "md"},
                    {"type": "text", "text": f"Cost : $ {COGS_UNIT:.2f}", "margin": "md"},
                    {"type": "text", "text": f"Profit : $ {GP_UNIT:.2f}", "margin": "md"},
                    # Add other elements here
                ],
            },
            # Add other elements here
        },
    }

    return flex_message


def reply_viz(text, reply_token):
    texts = text.split()
    query = {"type": texts[0]}

    plt.clf()

    plt.figure(figsize=(20, 8))
    if query["type"] == "Overview":
        sns.heatmap(data=df_2_pivot, annot=True, fmt=".0f", cmap="YlGnBu")
        plt.xlabel('DEPARTMENT_NAME')
        plt.ylabel('SITE_NAME')
        plt.title('%Sale Performance by DEPARTMENT_NAME and SITE_NAME')
        plt.savefig('Show_graph.png')
        plt.close()

    elif query["type"] == "สาขาสะพานควาย":
        # Graph 1 (Bar graph)
        plt.subplot(1, 2, 1)
        colors = plt.cm.Blues(top_10_brands['SALE_AMT'] / top_10_brands['SALE_AMT'].max())[::1]
        plt.bar(top_10_brands['BRAND'], top_10_brands['SALE_AMT'], color=colors)
        plt.xlabel('Brand')
        plt.ylabel('Total Sale Amount')
        plt.title('Top 10 Brands by Sale Amount in Sapankwai')
        plt.xticks(rotation=45, ha='right')

        # Graph 2 (Bubble heatmap)
        plt.subplot(1, 2, 2)
        scatter = plt.scatter(
            x=top_10_brands['PRICE'],
            y=top_10_brands['SALE_QTY'],
            c=top_10_brands['SALE_AMT'],
            cmap='Blues',
            s=top_10_brands['SALE_AMT'] * 0.01,
            alpha=0.7,
            edgecolors='black',
        )
        for i, txt in enumerate(top_10_brands['BRAND']):
            plt.annotate(txt, (top_10_brands['PRICE'].iloc[i], top_10_brands['SALE_QTY'].iloc[i]),
                         fontsize=10, ha='center', va='bottom')
        cbar = plt.colorbar(scatter)
        cbar.set_label('SALE_AMT', rotation=270, labelpad=15)
        plt.xlabel('Price')
        plt.ylabel('SALE_QTY')
        plt.title('Sales Performance by Brand for Sapankwai')

        # Save the figure for สาขาสะพานควาย
        plt.savefig('Show_graph.png')
        plt.close()

    elif query["type"] == "สาขาสำโรง":
        # Subplot for Graph 1
        plt.subplot(1, 2, 1)
        colors_graph1 = plt.cm.Greens(top_10_brands_samrong['SALE_AMT'] / top_10_brands_samrong['SALE_AMT'].max())[::1]
        plt.bar(top_10_brands_samrong['BRAND'], top_10_brands_samrong['SALE_AMT'], color=colors_graph1)
        plt.xlabel('Brand')
        plt.ylabel('Total Sale Amount')
        plt.title('Top 10 Brands by Sale Amount in Samrong')
        plt.xticks(rotation=45, ha='right')

        # Subplot for Graph 2
        plt.subplot(1, 2, 2)
        scatter_graph2 = plt.scatter(
            x=top_10_brands_samrong['PRICE'],
            y=top_10_brands_samrong['SALE_QTY'],
            c=top_10_brands_samrong['SALE_AMT'],
            cmap='Greens',
            s=top_10_brands_samrong['SALE_AMT'] * 0.01,
            alpha=0.7,
            edgecolors='black',
        )
        for i, txt in enumerate(top_10_brands_samrong['BRAND']):
            plt.annotate(txt, (top_10_brands_samrong['PRICE'].iloc[i], top_10_brands_samrong['SALE_QTY'].iloc[i]),
                         fontsize=10, ha='center', va='bottom')
        cbar_graph2 = plt.colorbar(scatter_graph2)
        cbar_graph2.set_label('SALE_AMT', rotation=270, labelpad=15)
        plt.xlabel('Price')
        plt.ylabel('SALE_QTY')
        plt.title('Sales Performance by Brand for Samrong')

        # Save the figure
        plt.savefig('Show_graph.png')
        plt.close()

    elif query["type"] == "สาขาสุขสวัสดิ์":
        # Subplot for Graph 1
        plt.subplot(1, 2, 1)
        colors_graph1 = plt.cm.Purples(top_10_brands_suksawat['SALE_AMT'] / top_10_brands_suksawat['SALE_AMT'].max())[::1]
        plt.bar(top_10_brands_suksawat['BRAND'], top_10_brands_suksawat['SALE_AMT'], color=colors_graph1)
        plt.xlabel('Brand')
        plt.ylabel('Total Sale Amount')
        plt.title('Top 10 Brands by Sale Amount in Suksawat')
        plt.xticks(rotation=45, ha='right')

        # Subplot for Graph 2
        plt.subplot(1, 2, 2)
        scatter_graph2 = plt.scatter(
            x=top_10_brands_suksawat['PRICE'],
            y=top_10_brands_suksawat['SALE_QTY'],
            c=top_10_brands_suksawat['SALE_AMT'],
            cmap='Purples',  # Use a predefined colormap name
            s=top_10_brands_suksawat['SALE_AMT'] * 0.01,
            alpha=0.7,
            edgecolors='black',
        )
        for i, txt in enumerate(top_10_brands_suksawat['BRAND']):
            plt.annotate(txt, (top_10_brands_suksawat['PRICE'].iloc[i], top_10_brands_suksawat['SALE_QTY'].iloc[i]),
                        fontsize=10, ha='center', va='bottom')
        cbar_graph2 = plt.colorbar(scatter_graph2)
        cbar_graph2.set_label('SALE_AMT', rotation=270, labelpad=15)
        plt.xlabel('Price')
        plt.ylabel('SALE_QTY')
        plt.title('Sales Performance by Brand for Suksawat')

        # Save the figure
        plt.savefig('Show_graph.png')
        plt.close()

    elif query["type"] == "สาขาพระราม2":
        # Subplot for Graph 1
        plt.subplot(1, 2, 1)
        colors_graph1 = plt.cm.YlOrBr(top_10_brands_rama2['SALE_AMT'] / top_10_brands_rama2['SALE_AMT'].max())[::1]
        plt.bar(top_10_brands_rama2['BRAND'], top_10_brands_rama2['SALE_AMT'], color=colors_graph1)
        plt.xlabel('Brand')
        plt.ylabel('Total Sale Amount')
        plt.title('Top 10 Brands by Sale Amount in Rama 2')
        plt.xticks(rotation=45, ha='right')

        # Subplot for Graph 2
        plt.subplot(1, 2, 2)
        scatter_graph2 = plt.scatter(
            x=top_10_brands_rama2['PRICE'],
            y=top_10_brands_rama2['SALE_QTY'],
            c=top_10_brands_rama2['SALE_AMT'],
            cmap='YlOrBr',  # Use a predefined colormap name
            s=top_10_brands_rama2['SALE_AMT'] * 0.01,
            alpha=0.7,
            edgecolors='black',
        )
        for i, txt in enumerate(top_10_brands_rama2['BRAND']):
            plt.annotate(txt, (top_10_brands_rama2['PRICE'].iloc[i], top_10_brands_rama2['SALE_QTY'].iloc[i]),
                        fontsize=10, ha='center', va='bottom')
        cbar_graph2 = plt.colorbar(scatter_graph2)
        cbar_graph2.set_label('SALE_AMT', rotation=270, labelpad=15)
        plt.xlabel('Price')
        plt.ylabel('SALE_QTY')
        plt.title('Sales Performance by Brand for Rama 2')

        # Save the figure
        plt.savefig('Show_graph.png')
        plt.close()

    elif query["type"] == "สาขาติวานนท์":
        # Subplot for Graph 1
        plt.subplot(1, 2, 1)
        colors_graph1_tiwanon = plt.cm.Reds(top_10_brands_tiwanon['SALE_AMT'] / top_10_brands_tiwanon['SALE_AMT'].max())[::1]
        plt.bar(top_10_brands_tiwanon['BRAND'], top_10_brands_tiwanon['SALE_AMT'], color=colors_graph1_tiwanon)
        plt.xlabel('Brand')
        plt.ylabel('Total Sale Amount')
        plt.title('Top 10 Brands by Sale Amount in Tiwanon')
        plt.xticks(rotation=45, ha='right')

        # Subplot for Graph 2
        plt.subplot(1, 2, 2)
        scatter_graph2_tiwanon = plt.scatter(
            x=top_10_brands_tiwanon['PRICE'],
            y=top_10_brands_tiwanon['SALE_QTY'],
            c=top_10_brands_tiwanon['SALE_AMT'],
            cmap='Reds',  # Use a predefined colormap name
            s=top_10_brands_tiwanon['SALE_AMT'] * 0.01,
            alpha=0.7,
            edgecolors='black',
        )
        for i, txt in enumerate(top_10_brands_tiwanon['BRAND']):
            plt.annotate(txt, (top_10_brands_tiwanon['PRICE'].iloc[i], top_10_brands_tiwanon['SALE_QTY'].iloc[i]),
                        fontsize=10, ha='center', va='bottom')
        cbar_graph2_tiwanon = plt.colorbar(scatter_graph2_tiwanon)
        cbar_graph2_tiwanon.set_label('SALE_AMT', rotation=270, labelpad=15)
        plt.xlabel('Price')
        plt.ylabel('SALE_QTY')
        plt.title('Sales Performance by Brand for Tiwanon')

        # Save the figure
        plt.savefig('Show_graph.png')
        plt.close()

    else:
        return "น้องจริงขอภัยค่ะ รบกวนกด 'Menu' ด้านล่างอีกครั้ง"

    uploaded = drive.CreateFile({'title': 'Show_graph.png'})
    uploaded.SetContentFile('Show_graph.png')
    uploaded.Upload()
    image_message = ImageSendMessage(
        original_content_url=uploaded["thumbnailLink"][:-5],
        preview_image_url=uploaded["thumbnailLink"]
        )
    line_bot_api.reply_message(reply_token,image_message)

# จริงใจช่วยตอบ Part

def reply_report(text, reply_token):
    texts = text.split()
    query = {"text": texts[0]}
    chk_error = 0

    if text == "ยอดขายวันนี้":
        query = """
            SELECT
            sum(SALE_AMT) Total_Sales
            FROM df
            where DATE_SALE ='2023-11-30'
            """
        query_result = ps.sqldf(query, globals()).to_dict()

        text_to_display = f"Total Sales:\n${query_result['Total_Sales'][0]:,.2f}"

    elif text == "ยอดขายเฉลี่ย":
        query = """
                    SELECT  AVG(AVG_SALE) AS AVG_SALE
                    FROM (
                          SELECT
                              SITE_NO,
                              SUM(SALE_AMT) AS AVG_SALE
                            FROM
                              df
                            WHERE
                              DATE_SALE ='2023-11-30'
                            GROUP BY
                              SITE_NO
                      ) AS Subquery;
                     """

        query_result = ps.sqldf(query, globals()).to_dict()
        text_to_display = f"Avg Sales :\n${query_result['AVG_SALE'][0]:,.2f}"

    elif text == "ยอดขายสูงสุด":
        query = """
                  SELECT
                  SITE_NO,SITE_NAME,
                  MAX(Sum_Sales) AS Total_Sales
                FROM (
                  SELECT
                    SITE_NO,SITE_NAME,
                    SUM(SALE_AMT) AS Sum_Sales
                  FROM
                    df
                  WHERE
                    DATE_SALE ='2023-11-30'
                  GROUP BY
                    SITE_NO,SITE_NAME
                ) AS Subquery;
                  """

        query_result = ps.sqldf(query, globals()).to_dict()
        text_to_display = f"Store: {query_result['SITE_NO'][0]}\nStore Name: {query_result['SITE_NAME'][0]}\nTotal Sales: ${query_result['Total_Sales'][0]:,.2f}"

    elif text == "ยอดขายน้อยสุด":
        query = """
                  SELECT
                  SITE_NO,SITE_NAME,
                  MIN(Sum_Sales) AS Total_Sales
                FROM (
                  SELECT
                    SITE_NO,SITE_NAME,
                    SUM(SALE_AMT) AS Sum_Sales
                  FROM
                    df
                  WHERE
                    DATE_SALE ='2023-11-30'
                  GROUP BY
                    SITE_NO,SITE_NAME
                ) AS Subquery;
                  """

        query_result = ps.sqldf(query, globals()).to_dict()
        text_to_display = f"Store: {query_result['SITE_NO'][0]}\nStore Name: {query_result['SITE_NAME'][0]}\nTotal Sales: ${query_result['Total_Sales'][0]:,.2f}"

    elif text == "ต้นทุนวันนี้":
        query = """
                  SELECT
                  sum(COGS_AMT) Total_Cost
                  FROM df
                  where DATE_SALE ='2023-11-30'
                  """

        query_result = ps.sqldf(query, globals()).to_dict()
        text_to_display = f"Total Cost:\n${query_result['Total_Cost'][0]:,.2f}"

    elif text == "ต้นทุนเฉลี่ย":
        query = """
                 SELECT  AVG(COGS_AMT) AS COGS_AMT
                 FROM (
                      SELECT
                          SITE_NO,
                          SUM(COGS_AMT) AS COGS_AMT
                        FROM
                          df
                        WHERE
                          DATE_SALE = '2023-11-30'
                        GROUP BY
                          SITE_NO
                   ) AS Subquery;
                  """

        query_result = ps.sqldf(query, globals()).to_dict()
        text_to_display =f"Avg Cost :\n${query_result['COGS_AMT'][0]:,.2f}"

    elif text == "สินค้าที่ต้นทุนสูงที่สุด":
        query = """
                  SELECT  ART_NAME ,
                  max(COGS_AMT) COGS_AMT
                 FROM (
                      SELECT
                          ART_NAME,
                          SUM(COGS_AMT) AS COGS_AMT
                        FROM
                          df
                        WHERE
                          DATE_SALE = '2023-11-30'
                        GROUP BY
                          ART_NAME
                   ) AS Subquery;
                  """

        query_result = ps.sqldf(query, globals()).to_dict()
        text_to_display =f"Product: {query_result['ART_NAME'][0]}\nTotal Cost: ${query_result['COGS_AMT'][0]:,.2f}"

    elif text == "สินค้าที่ต้นทุนต่ำที่สุด":
        query = """
                  SELECT  ART_NAME ,
                  MIN(COGS_AMT) COGS_AMT
                 FROM (
                      SELECT
                          ART_NAME,
                          SUM(COGS_AMT) AS COGS_AMT
                        FROM
                          df
                        WHERE
                          DATE_SALE = '2023-11-30'
                          AND  COGS_AMT > 0
                        GROUP BY
                          ART_NAME
                   ) AS Subquery;
                  """

        query_result = ps.sqldf(query, globals()).to_dict()
        text_to_display =f"Product : {query_result['ART_NAME'][0]}\nTotal Cost: ${query_result['COGS_AMT'][0]:,.2f}"

    elif text == "กำไรวันนี้":
        query = """
                  SELECT
                  sum(GP_AMT) Total_Profit
                  FROM df
                  where DATE_SALE ='2023-11-30'
                  """

        query_result = ps.sqldf(query, globals()).to_dict()
        text_to_display =f"Total Profit :\n${query_result['Total_Profit'][0]:,.2f}"

    elif text == "กำไรเฉลี่ย":
        query = """
                SELECT  AVG(GP_AMT) AS AVG_GP_AMT
                FROM (
                      SELECT
                          SITE_NO,
                          SUM(GP_AMT) AS GP_AMT
                        FROM
                          df
                        WHERE
                          DATE_SALE = '2023-11-30'
                        GROUP BY
                          SITE_NO
                   )   AS Subquery;
                  """

        query_result = ps.sqldf(query, globals()).to_dict()
        text_to_display = f"Avg Profits :\n${query_result['AVG_GP_AMT'][0]:,.2f}"

    elif text == "กำไรสูงสุดและต่ำสุด":
        query = """
                WITH MaxMinGP AS (
                  SELECT
                    SITE_NO,
                    SITE_NAME,
                    SUM(GP_AMT) AS totalGP_AMT
                  FROM
                    df
                  WHERE
                    DATE_SALE ='2023-11-30'
                  GROUP BY
                    SITE_NO, SITE_NAME
                )

                SELECT
                  SITE_NO,
                  SITE_NAME,
                  'MAX PRO' AS ma,
                  totalGP_AMT AS GP_AMT
                FROM
                  MaxMinGP
                WHERE
                  totalGP_AMT = (SELECT MAX(totalGP_AMT) FROM MaxMinGP)

                UNION ALL

                SELECT
                  SITE_NO,
                  SITE_NAME,
                  'MAX PRO' AS ma,
                  totalGP_AMT AS GP_AMT
                FROM
                  MaxMinGP
                WHERE
                  totalGP_AMT = (SELECT MIN(totalGP_AMT) FROM MaxMinGP);
                  """

        query_result = ps.sqldf(query, globals()).to_dict()
        text_to_display1 = f"Store: {query_result['SITE_NO'][0]}\nStore Name: {query_result['SITE_NAME'][0]}\nMax Profits: ${query_result['GP_AMT'][0]:,.2f}\n"
        text_to_display2 = f"Store: {query_result['SITE_NO'][1]}\nStore Name: {query_result['SITE_NAME'][1]}\nMIN Profits: ${query_result['GP_AMT'][1]:,.2f}"
        text_to_display = text_to_display1 + "\n" + text_to_display2

    elif text == "สินค้าที่ทำกำไรสูงสุด":
        query = """
                  SELECT ART_NAME ,
                  max(GP_AMT) maxGP_AMT
                  FROM df
                  where DATE_SALE ='2023-11-30'
                  """

        query_result = ps.sqldf(query, globals()).to_dict()
        text_to_display =f"Product: {query_result['ART_NAME'][0]}\nTotal Profit: ${query_result['maxGP_AMT'][0]:,.2f}"

    else:
        chk_error = 1
        text_to_display = "น้องจริงใจขออภัย เกินความสามารถของน้องจริงใจขอรับ"
        text = "น้องจริงใจขออภัย เกินความสามารถของน้องจริงใจขอรับ"

    if chk_error == 1:
        reply_text = f"น้องจริงใจขออภัย เกินความสามารถของน้องจริงใจขอรับ"
    else:
        reply_text = text_to_display

    text_message = TextSendMessage(text=reply_text)
    line_bot_api.reply_message(reply_token,text_message)

#Flask
if __name__ == '__main__':
    public_url_s = public_url.replace("http://","https://")
    print(f"To acces the Gloable link please click {public_url_s}/callback")
    print("Starting app on port %d" % port_no)
    app.run(debug=False, port=port_no, threaded=True)

To acces the Gloable link please click https://3bfa-34-106-108-116.ngrok-free.app/callback
Starting app on port 5000
 * Serving Flask app '__main__'
 * Debug mode: off


<ipython-input-132-d6afbed79a61>:29: LineBotSdkDeprecatedIn30: Call to deprecated class LineBotApi. (Use v3 class; linebot.v3.<feature>. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api = LineBotApi('Z0Qqy1sWUMw5RNKsaChGQKLSHwkivRGSKr4KYKFbcGp8TLb96q2TBQ6ycIqp937tG//hpOFL8qwRqMUlZE86XEkPg5f4SU59Wa92xMD4bZVPiPdYbwMaFwTOSA4yF0Bz0N4SiEZR99iOCtKAAR3w8wdB04t89/1O/w1cDnyilFU=')
<ipython-input-132-d6afbed79a61>:30: LineBotSdkDeprecatedIn30: Call to deprecated class WebhookHandler. (Use 'from linebot.v3.webhook import WebhookHandler' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  handler = WebhookHandler('2ace8bbe1a2dc37fde3cb4593d12c1b8')
INFO:werkzeug:WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
INFO:werkze

id = U890896db790d22c3f877fd71409bf6f1
name = OIL♡
text = Jan
intent = jingjai Target-Actual-custom
reply_token = bf16eff98e834f1395905d0605caa860
Response Data: {'month': 'Jan', ' sale_target ': '5,894,729.03', ' total_actual ': '5,319,276.20', '%Achieve': '90%'}
Flex Message Content: {'type': 'flex', 'altText': 'Target vs Actual', 'contents': {'type': 'bubble', 'hero': {'type': 'image', 'url': 'https://img.freepik.com/free-vector/hand-drawn-business-strategy_23-2149164272.jpg', 'size': 'full', 'aspectRatio': '20:13', 'aspectMode': 'cover'}, 'body': {'type': 'box', 'layout': 'vertical', 'contents': [{'type': 'text', 'text': 'Target vs Actual: Jan', 'weight': 'bold', 'size': 'xl'}, {'type': 'text', 'text': 'Sales Target: 5,894,729.03', 'margin': 'md'}, {'type': 'text', 'text': 'Total Actual: 5,319,276.20', 'margin': 'md'}, {'type': 'text', 'text': '% Achieve: 90.24%', 'margin': 'md'}]}}}
Complete FlexSendMessage: {"altText": "Target vs Actual", "contents": {"body": {"contents": [{"size

<ipython-input-132-d6afbed79a61>:72: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(reply_token, flex_message)
INFO:werkzeug:127.0.0.1 - - [08/Dec/2023 19:18:05] "POST /callback HTTP/1.1" 200 -


id = U890896db790d22c3f877fd71409bf6f1
name = OIL♡
text = กำไรสูงสุดและต่ำสุด
intent = jingjai qa - custom
reply_token = 71718ac92cda4e36ab2ff4b8d4cefecd


...

id = U890896db790d22c3f877fd71409bf6f1
name = OIL♡
text = ต้นทุนเฉลี่ย
intent = jingjai qa - custom
reply_token = 7d2b815ec27548af96d683490a63816e


<ipython-input-132-d6afbed79a61>:687: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(reply_token,text_message)
INFO:werkzeug:127.0.0.1 - - [08/Dec/2023 19:18:47] "POST /callback HTTP/1.1" 200 -
<ipython-input-132-d6afbed79a61>:40: LineBotSdkDeprecatedIn30: Call to deprecated method get_profile. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).get_profile(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  disname = line_bot_api.get_profile(id).display_name


id = U890896db790d22c3f877fd71409bf6f1
name = OIL♡
text = สาขาติวานนท์
intent = jingjai summary - custom
reply_token = e558a99a8a954bf8a68dabccb076f802


<ipython-input-132-d6afbed79a61>:440: LineBotSdkDeprecatedIn30: Call to deprecated method reply_message. (Use 'from linebot.v3.messaging import MessagingApi' and 'MessagingApi(...).reply_message(...)' instead. See https://github.com/line/line-bot-sdk-python/blob/master/README.rst for more details.) -- Deprecated since version 3.0.0.
  line_bot_api.reply_message(reply_token,image_message)
INFO:werkzeug:127.0.0.1 - - [08/Dec/2023 19:19:19] "POST /callback HTTP/1.1" 200 -


<Figure size 640x480 with 0 Axes>